# Музыкальный стриминговый сервис

На данных музыкального стримингого сервиса необходимо сравнить поведение пользователей двух столиц: Москвы и Санкт-Петербурга.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей в зависимости от дня недели. В разных городах по-разному. 
2. Преобладание жанров в понедельник утром в Москве и Питере. А также преобладание жанров в пятницу вечером в зависимости от города.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

1. Обзор данных.
2. Предобработка данных.
3. Проверка гипотез.

## Обзор данных

Знакомство с данными сервиса.

In [1]:
# импорт библиотеки pandas
import pandas as pd 

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/yandex_music_project.csv')

In [3]:
# получение первых 10 строк таблицы df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Отсутствует змеиный регистр.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

При знакомстве с данными было отмечено следующее:

Строка - данные конкретного прослушенного трека. Столбцы - данные и о треке (название, исполнитель, жанр), и о прослушавшем его пользователе (id, город, время и день прослушивания).

На первый взгляд все данные в норме, но есть некоторые недочёты: нарушение стиля, пропуски, возможные дубликаты. Устраню эти проблемы для проведения дальнейшей работы с данными.

## Предобработка данных

- Исправление стиля в заголовках столбцов 
- Работа с пропусками 
- Проверка данных на дубликаты.

### Заголовки


In [5]:
# перечень названий столбцов таблицы df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [6]:
# переименование столбцов
df = df.rename(columns={'  userID':'user_id', 'Track':'track','  City  ':'city', 'Day':'day'})

In [7]:
# проверка результатов - перечень названий столбцов
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

In [8]:
# подсчёт пропусков
counting_null = df.isna().sum()
print(counting_null)

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64


Заменяю пропущенные значения в столбцах 'track', 'artist' и 'genre' на особое значение 'unknown', так как не могу подобрать точное значение.

In [9]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track','artist','genre']
for i in columns_to_replace:
    df[i] = df[i].fillna('unknown')

In [10]:
# подсчёт пропусков
counting_null = df.isna().sum()
print(counting_null)

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Дубликаты

In [11]:
# подсчёт явных дубликатов
counting_duplicates = df.duplicated().sum()
print(counting_duplicates)

3826


In [12]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop = True)

In [13]:
# проверка на отсутствие дубликатов
counting_duplicates = df.duplicated().sum()
print(counting_duplicates)

0


Проверяю столбец 'genre' на наличие дубликатов, так как название одного и того же жанра может быть написано по-разному, приведу все к одинаковым значениям.

In [14]:
# Просмотр уникальных названий жанров
genres = df['genre'].sort_values().unique()
print(genres)

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

In [15]:
# Функция для замены неявных дубликатов
def replace_wrong_genres(wrong_genres, correct_genre):
    df['genre'] = df['genre'].replace(wrong_genres, correct_genre)
    return

In [16]:
# Устранение неявных дубликатов
replace_wrong_genres('hip', 'hiphop')
replace_wrong_genres('hop', 'hiphop')
replace_wrong_genres('hip-hop', 'hiphop')
replace_wrong_genres('электроника', 'electronic')
replace_wrong_genres('ïîï', 'pop')

In [17]:
# Проверка на неявные дубликаты
genres = df['genre'].sort_values().unique()
print(genres)

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

**Выводы**

Решены три основные проблемы в предобработке данных:
- неверный стиль названия столбцов приведен к корректному
- устранены пропуски 
- устранены дубликаты

Теперь данные готовы для проверки поставленных гипотез

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Гипотеза 1: _Активность пользователей в зависимости от дня недели. В разных городах по-разному._

* Проверю данное предположение по данным о трёх днях недели - понедельнике, среде и пятнице
* Поделю выборку на две по городу пользователя: Москва и Санкт-Петербург
* Сравню количество прослушиваний каждой группу в каждый обозначенный день недели

In [18]:
# Подсчёт прослушиваний в каждом городе
population_count = df.groupby('city')['user_id'].count()
print(population_count) 

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64


In [19]:
# Подсчёт прослушиваний в каждый из трёх дней
days_count = df.groupby('day')['user_id'].count()
print(days_count)

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

На первый взгляд прослушиваний в Москве почти в 2,5 раза больше, чем в Санкт-Петербурге, но это может быть не из-за частого прослушивания именно жителями Москвы, а из-за того, что в столице в принципе население больше, чем в Санкт-Петербурге.

Также при сравнении количества прослушиваний в конкретный день недели вижу, что пятница и понедельник - самые активные дни. Но при делении выборки по городам это может измениться.

Создам функцию для подсчета прослушиваний для конкретного дня и города.

In [20]:
# создание функции number_tracks()
# два параметрами: day, city.
def number_tracks(day, city):
    track_list_day = df[df['day'] == day]
    track_list = track_list_day[track_list_day['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [21]:
# количество прослушиваний в Москве по понедельникам
number_tracks('Monday', 'Moscow')

15740

In [22]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks('Monday', 'Saint-Petersburg')

5614

In [23]:
# количество прослушиваний в Москве по средам
number_tracks('Wednesday', 'Moscow')

11056

In [24]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [25]:
# количество прослушиваний в Москве по пятницам
number_tracks('Friday', 'Moscow')

15945

In [26]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks('Friday', 'Saint-Petersburg')

5895

Создаю новую таблицу table, где столбцы - город, понедельник, среда, пятница, а строки - данные по Москве и Санкт-Петербургу, полученные с помощью функции number_tracks.

In [27]:
# Таблица с результатами
data = [['Moscow,', 
         number_tracks('Monday', 'Moscow'), 
         number_tracks('Wednesday', 'Moscow'), 
         number_tracks('Friday', 'Moscow')],
        ['Saint-Petersburg', 
         number_tracks('Monday', 'Saint-Petersburg'), 
         number_tracks('Wednesday', 'Saint-Petersburg'), 
         number_tracks('Friday', 'Saint-Petersburg')]]
columns = ['city', 'monday', 'wednesday', 'friday']
table = pd.DataFrame(data=data, columns=columns)
display(table)

,city,monday,wednesday,friday
0,"Moscow,",15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные подтверждают верность первой гипотезы.

### Музыка в начале и в конце недели

Гипотеза 2: _Преобладание жанров в понедельник утром в Москве и Питере. А также преобладание жанров в пятницу вечером в зависимости от города._

Создаю два датафрейма: срезы по городам

In [28]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [29]:
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

Создаю функцию `genre_weekday()`, которая должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [30]:
def genre_weekday(table, day, time1, time2):
#   table - датафрейм, 
#   day - значение аргумента day
#   time больше значения аргумента time1
#   time меньше значения аргумента time2
    genre1 = table[table['day'] == day]
    genre2 = genre1[genre1['time'] > time1]
    genre_df = genre2[genre2['time'] < time2]
#   сгруппирую датафрейм genre_df по столбцу genre и посчитаю количество

    genre_df_count = genre_df.groupby('genre')['user_id'].count()
#   отсортирую genre_df_count по убыванию встречаемости и сохраню
#   в переменную genre_df_sorted
    genre_df_sorted = genre_df_count.sort_values(ascending = False)

    return genre_df_sorted.head(10)

Cравнение результатов функции для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [31]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')


genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64

In [32]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64

In [33]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: user_id, dtype: int64

In [34]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: user_id, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза 3: _Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп._

In [35]:
# группировка таблицы moscow_general по столбцу 'genre', 
moscow_genres = moscow_general.groupby('genre')['user_id'].count()
moscow_genres = moscow_genres.sort_values(ascending = False)

In [36]:
# просмотр первых 10 строк moscow_genres
print(moscow_genres.head(10))

genre
pop            5893
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: user_id, dtype: int64


In [37]:
# группировка таблицы spb_general по столбцу 'genre', 
spb_genres = spb_general.groupby('genre')['user_id'].count()
spb_genres = spb_genres.sort_values(ascending = False)

In [38]:
# просмотр первых 10 строк spb_genres
print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: user_id, dtype: int64


**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

1. **Активность пользователей в зависимости от дня недели. В разных городах по-разному.**

*Гипотеза 1 подтвердилась*


2. **Преобладание жанров в понедельник утром в Москве и Питере. А также преобладание жанров в пятницу вечером в зависимости от города** 

*Гипотеза 2 подтвердилась частично. Из-за пропусков в данных результат мог быть другим.*


3. **Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.**
*Гипотеза 3 вовсе не подвердилась. Существующие различия не заметны на основной массе пользователей.*